<a href="https://colab.research.google.com/github/ja-yu/AI/blob/main/NeuralNetwork_ch05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/oreilly-japan/deep-learning-from-scratch.git

Cloning into 'deep-learning-from-scratch'...
remote: Enumerating objects: 446, done.
remote: Total 446 (delta 0), reused 0 (delta 0), pack-reused 446
Receiving objects: 100% (446/446), 5.52 MiB | 29.57 MiB/s, done.
Resolving deltas: 100% (232/232), done.


In [2]:
ls

deep-learning-from-scratch/  sample_data/


In [3]:
cd deep-learning-from-scratch/ 

/content/deep-learning-from-scratch


In [4]:
ls

ch01/  ch03/  ch05/  ch07/  common/   LICENSE.md*
ch02/  ch04/  ch06/  ch08/  dataset/  README.md


In [5]:
cd ch05/

/content/deep-learning-from-scratch/ch05


In [6]:
ls

buy_apple_orange.py  gradient_check.py  train_neuralnet.py
buy_apple.py         layer_naive.py     two_layer_net.py


In [7]:
cat buy_apple_orange.py

# coding: utf-8
from layer_naive import *

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)


In [8]:
! python buy_apple_orange.py

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


In [9]:
cat buy_apple.py 

# coding: utf-8
from layer_naive import *


apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)


In [10]:
! python buy_apple.py 

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


In [11]:
cat gradient_check.py

# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

In [13]:
! python gradient_check.py

Traceback (most recent call last):
  File "gradient_check.py", line 16, in <module>
    grad_numerical = network.numerical_gradient(x_batch, t_batch)
  File "/content/deep-learning-from-scratch/ch05/two_layer_net.py", line 52, in numerical_gradient
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
  File "../common/gradient.py", line 46, in numerical_gradient
    fxh2 = f(x) # f(x-h)
  File "/content/deep-learning-from-scratch/ch05/two_layer_net.py", line 49, in <lambda>
    loss_W = lambda W: self.loss(x, t)
  File "/content/deep-learning-from-scratch/ch05/two_layer_net.py", line 36, in loss
    y = self.predict(x)
  File "/content/deep-learning-from-scratch/ch05/two_layer_net.py", line 30, in predict
    x = layer.forward(x)
  File "../common/layers.py", line 57, in forward
    out = np.dot(self.x, self.W) + self.b
  File "<__array_function__ internals>", line 4, in dot
  File "/usr/local/lib/python3.6/dist-packages/numpy/core/multiarray.py", line 707, in dot
    @array

In [14]:
cat layer_naive.py

# coding: utf-8


class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy


class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy


In [15]:
! python layer_naive.py

In [16]:
cat train_neuralnet.py

# coding: utf-8
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 勾配
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % it

In [17]:
! python train_neuralnet.py

0.13063333333333332 0.1286
0.9035 0.9066
0.9219333333333334 0.9257
0.9344166666666667 0.9348
0.9436333333333333 0.9415
0.9506333333333333 0.9495
0.9548166666666666 0.9529
0.9598166666666667 0.9576
0.9642833333333334 0.9605
Traceback (most recent call last):
  File "train_neuralnet.py", line 32, in <module>
    grad = network.gradient(x_batch, t_batch)
  File "/content/deep-learning-from-scratch/ch05/two_layer_net.py", line 70, in gradient
    dout = layer.backward(dout)
  File "../common/layers.py", line 62, in backward
    dx = np.dot(dout, self.W.T)
  File "<__array_function__ internals>", line 6, in dot
KeyboardInterrupt


In [18]:
cat two_layer_net.py

# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.laye

In [19]:
! python two_layer_net.py